# Post-processing of micromagnetic simulations in MuMax³: Nanotrack and Nanodisk



# -> Plotting the trajectory of the skyrmion


By means of $mumax3-convert -comp z -color "blue,white,red" -png -o*.ovf$, the ovf file is converted into a png file, whereby will be used to plot the trajectory of the center of mass of skyrmion throughout the nanotrack. In this case, I chose the color blue to represent the material and red and white to represent the skyrmion nucleus. Furthermore, it is possible to make a video of the simulation by using $imshow$.

In [ ]:
import cv2
import glob 
from natsort import natsorted

#input_folder = '/home/jfelipe/TCC/racetrack_memory.out/*.png' 
#input_folder = '/home/jfelipe/TCC/racetrack_memory.out/*.png' 
#output_video = 'skyrmion_nanotrack_video.mp4'
#fps = 30 # frames per second of the output video


image_files = natsorted(glob.glob(input_folder)) # Using natsorted to sort the files in natural order (img 1 , img2 , etc)

class SimulationVideoCreator():
    def __init__(self, input_folder, output_video, fps=30):
        self.input_folder = input_folder
        self.output_video = output_video
        self.fps = fps

    def sort_image_files(self):
        image_files = natsorted(glob.glob(self.input_folder))
        return image_files
    
    def create_video(self):
        first_frame = cv2.imread(self.sort_image_files()[0])
        height, width, layers = first_frame.shape
        video_writer = cv2.VideoWriter(self.output_video, cv2.VideoWriter_fourcc(*'mp4v'), self.fps, (width, height)) #fourcc code for mp4 format (fourcharacter code)

        for image_file in self.sort_image_files():
            frame = cv2.imread(image_file)
            video_writer.write(frame)
        video_writer.release() # Release the video writer to finalize the video file


if __name__ == "__main__":
    video_creator = SimulationVideoCreator('/home/jfelipe/TCC/racetrack_memory.out/*.png', 'skyrmion_nanotrack_video.mp4', fps=30)
    video_creator.create_video()